<a href="https://colab.research.google.com/github/yunyoungwoo/2024S-Ajou-ML-FP/blob/main/CNN_balanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from torchvision import transforms
import torchvision.transforms.functional as v2
from tqdm import tqdm
import numpy as np
import os
import torch.nn.functional as F
import torch.nn as nn
from PIL import Image
import pickle
from sklearn.metrics import f1_score

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
# 데이터프레임을 사용하여 커스텀 데이터셋 생성
class BaseDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, transform):
        self.df = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_path = self.df.iloc[index]['image_path']  # 이미지 파일 경로 컬럼 이름이 'image_path'라고 가정
        img = Image.open(img_path).convert("RGB")
        transformed_img = self.transform(img)

        # 원-핫 인코딩된 클래스 레이블을 가져옴
        class_labels = self.df.iloc[index][['class_0', 'class_1', 'class_2', 'class_3']].values.astype(int)
        class_id = torch.argmax(torch.tensor(class_labels)).item()  # 정수형 레이블로 변환

        return transformed_img, class_id

In [49]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),  # 랜덤 리사이즈 및 자르기
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),  # 컬러 지터
    transforms.RandomRotation(degrees=90),  # 랜덤 회전
    transforms.RandomHorizontalFlip(p=0.5),  # 랜덤 수평 뒤집기
    transforms.RandomVerticalFlip(p=0.5),  # 랜덤 수직 뒤집
    transforms.ToTensor(),  # torch 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
]) # 하이퍼파라미터

In [50]:
# 저장된 데이터프레임 불러오기
with open('/content/drive/MyDrive/Colab Notebooks/FP/training_df.pkl', 'rb') as f:
    training_df = pickle.load(f)

with open('/content/drive/MyDrive/Colab Notebooks/FP/validation_df.pkl', 'rb') as f:
    validation_df = pickle.load(f)

with open('/content/drive/MyDrive/Colab Notebooks/FP/test_df.pkl', 'rb') as f:
    test_df = pickle.load(f)

training_df.head()

,image_path,class_0,class_1,class_2,class_3
0,/content/drive/MyDrive/Colab Notebooks/FP/Trai...,False,False,False,True
1,/content/drive/MyDrive/Colab Notebooks/FP/Trai...,False,False,False,True
2,/content/drive/MyDrive/Colab Notebooks/FP/Trai...,False,False,False,True
3,/content/drive/MyDrive/Colab Notebooks/FP/Trai...,False,False,False,True
4,/content/drive/MyDrive/Colab Notebooks/FP/Trai...,False,False,False,True


In [51]:
# 데이터프레임을 사용하여 데이터셋 생성
train_dataset = BaseDataset(training_df, transform=train_transforms)
val_dataset = BaseDataset(validation_df, transform=train_transforms)
test_dataset = BaseDataset(test_df, transform=train_transforms)

In [53]:
# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [54]:
class ImprovedCNN(nn.Module):
    def __init__(self, num_classes=4):  # 클래스 수를 인자로 받도록 수정
        super(ImprovedCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=64 * 14 * 14, out_features=1024)  # Fully Connected Layer 크기 수정
        self.fc2 = nn.Linear(in_features=1024, out_features=128)  # Fully Connected Layer 크기 줄이기
        self.fc3 = nn.Linear(in_features=128, out_features=num_classes)  # 클래스 수에 맞게 수정
        self.dropout = nn.Dropout(p=0.25)
        self.batchnorm1 = nn.BatchNorm2d(8)
        self.batchnorm2 = nn.BatchNorm2d(16)
        self.batchnorm3 = nn.BatchNorm2d(32)
        self.batchnorm4 = nn.BatchNorm2d(64)

    def forward(self, x):
        x = F.relu(self.batchnorm1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.batchnorm2(self.conv2(x)))
        x = self.pool(x)
        x = F.relu(self.batchnorm3(self.conv3(x)))
        x = self.pool(x)
        x = F.relu(self.batchnorm4(self.conv4(x)))
        x = self.pool(x)

        x = x.view(x.size(0), -1)  # 배치 크기에 맞게 변환
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)

        return x

In [55]:
# 모델 생성 및 CUDA 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ImprovedCNN().to(device)

In [56]:
# 손실 함수 및 최적화 함수, 스케쥴러 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.001)  # AdamW 옵티마이저
lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)  # CosineAnnealingLR 학습률 스케줄러

In [57]:
def train(train_loader, model, criterion, optimizer, lr_scheduler):
    total_samples = 0
    num_batches = len(train_loader)
    model.train()
    epoch_loss = 0
    epoch_accuracy = 0
    all_predictions = []
    all_targets = []

    for i, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)  # 이미지 경로 제거
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        _, predictions = outputs.max(1)
        all_predictions.extend(predictions.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())
        epoch_accuracy += torch.sum(predictions == targets).item()
        total_samples += targets.size(0)

    lr_scheduler.step()

    accuracy = epoch_accuracy / total_samples
    macro_f1 = f1_score(all_targets, all_predictions, average='macro')
    return accuracy, epoch_loss / num_batches, macro_f1

In [58]:
def test(val_loader, model, criterion):
    total_samples = 0
    num_batches = len(val_loader)
    epoch_loss = 0
    epoch_accuracy = 0
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        model.eval()
        for i, (data, targets) in enumerate(val_loader):
            data = data.to(device)
            targets = targets.to(device)  # 이미지 경로 제거
            outputs = model(data)
            loss = criterion(outputs, targets)
            epoch_loss += loss.item()
            _, predictions = outputs.max(1)
            all_predictions.extend(predictions.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            epoch_accuracy += torch.sum(predictions == targets).item()
            total_samples += targets.size(0)

    accuracy = epoch_accuracy / total_samples
    macro_f1 = f1_score(all_targets, all_predictions, average='macro')
    return accuracy, epoch_loss / num_batches, macro_f1

In [59]:
# 학습 및 검증 실행
EPOCHS = 5 #하이퍼파라미터
patience = 10
counter = 0
best_loss = np.inf
logs = {'train_loss': [], 'train_acc': [], 'train_f1': [], 'val_loss': [], 'val_acc': [], 'val_f1': []}

In [60]:
from tqdm import tqdm

for epoch in tqdm(range(EPOCHS)):
    train_acc, train_loss, train_f1 = train(train_loader, model, criterion, optimizer, lr_scheduler)
    val_acc, val_loss, val_f1 = test(val_loader, model, criterion)
    print(f'epoch: {epoch} \
    train_loss = {train_loss:.4f}, train_acc: {train_acc:.4f}, train_f1: {train_f1:.4f} \
    val_loss = {val_loss:.4f}, val_acc: {val_acc:.4f}, val_f1: {val_f1:.4f} \
    learning rate: {optimizer.param_groups[0]["lr"]}')
    logs['train_loss'].append(train_loss)
    logs['train_acc'].append(train_acc)
    logs['train_f1'].append(train_f1)
    logs['val_loss'].append(val_loss)
    logs['val_acc'].append(val_acc)
    logs['val_f1'].append(val_f1)

    if val_loss < best_loss:
        counter = 0
        best_loss = val_loss
        torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/FP/Improved_CNN_best.pth")
    else:
        counter += 1
    if counter >= patience:
        test_acc, val_loss, test_f1 = test(test_loader, model, criterion)
        print("Early stop!")
        print(f"Test accuracy: {test_acc}, Test f1: {test_f1}")
        break


  0%|          | 0/25 [00:00<?, ?it/s]

epoch: 0     train_loss = 1.3483, train_acc: 0.3335, train_f1: 0.3262     val_loss = 1.2074, val_acc: 0.4300, val_f1: 0.4124     learning rate: 9.755282581475769e-05


  8%|▊         | 2/25 [04:17<49:32, 129.23s/it]

epoch: 1     train_loss = 1.2161, train_acc: 0.4095, train_f1: 0.3906     val_loss = 1.1760, val_acc: 0.4600, val_f1: 0.4454     learning rate: 9.045084971874737e-05


 12%|█▏        | 3/25 [06:24<47:04, 128.41s/it]

...

epoch: 21     train_loss = 1.0807, train_acc: 0.5110, train_f1: 0.4988     val_loss = 1.0412, val_acc: 0.5067, val_f1: 0.4917     learning rate: 9.04508497187478e-05
epoch: 22     train_loss = 1.0664, train_acc: 0.5045, train_f1: 0.4898     val_loss = 1.0242, val_acc: 0.5067, val_f1: 0.4859     learning rate: 7.938926261462401e-05


 92%|█████████▏| 23/25 [50:29<04:26, 133.45s/it]

epoch: 23     train_loss = 1.0695, train_acc: 0.5110, train_f1: 0.5041     val_loss = 1.0226, val_acc: 0.5000, val_f1: 0.4593     learning rate: 6.545084971874767e-05


100%|██████████| 25/25 [55:06<00:00, 132.26s/it]

epoch: 24     train_loss = 1.0563, train_acc: 0.5270, train_f1: 0.5187     val_loss = 1.0426, val_acc: 0.5100, val_f1: 0.4993     learning rate: 5.0000000000000226e-05
